In [1]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib.inline
import warnings
warnings.filterwarnings(action='ignore')
import optuna
from sklearn.ensemble import HistGradientBoostingRegressor
from category_encoders import CatBoostEncoder
from sklearn.model_selection import train_test_split, cross_val_score,KFold



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/playground-series-s3e19/sample_submission.csv
/kaggle/input/playground-series-s3e19/train.csv
/kaggle/input/playground-series-s3e19/test.csv


In [2]:
train=pd.read_csv('/kaggle/input/playground-series-s3e19/train.csv')

In [3]:
test = pd.read_csv('/kaggle/input/playground-series-s3e19/test.csv')

In [4]:
test.head()

,id,date,country,store,product
0,136950,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding
1,136951,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs
2,136952,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People
3,136953,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions
4,136954,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136950 entries, 0 to 136949
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        136950 non-null  int64 
 1   date      136950 non-null  object
 2   country   136950 non-null  object
 3   store     136950 non-null  object
 4   product   136950 non-null  object
 5   num_sold  136950 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 6.3+ MB


In [6]:
train.isnull().sum()

id          0
date        0
country     0
store       0
product     0
num_sold    0
dtype: int64

In [7]:
train['date']=pd.to_datetime(train['date'])

In [8]:
train.head()

,id,date,country,store,product,num_sold
0,0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63
1,1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66
2,2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9
3,3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59
4,4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49


In [9]:
train.isnull().sum()

id          0
date        0
country     0
store       0
product     0
num_sold    0
dtype: int64

Add features & holiday column for better prediction

In [10]:
train.describe()

,id,num_sold
count,136950.00000,136950.000000
mean,68474.50000,165.522636
std,39534.20402,183.691575
min,0.00000,2.000000
25%,34237.25000,46.000000
50%,68474.50000,98.000000
75%,102711.75000,184.000000
max,136949.00000,1380.000000


In [11]:
for col in train.columns:
    print(f"{col} has {len(train[col].unique())} values")

id has 136950 values
date has 1826 values
country has 5 values
store has 3 values
product has 5 values
num_sold has 1028 values


In [12]:
products= train['product'].unique()

In [13]:
products

array(['Using LLMs to Improve Your Coding',
       'Using LLMs to Train More LLMs',
       'Using LLMs to Win Friends and Influence People',
       'Using LLMs to Win More Kaggle Competitions',
       'Using LLMs to Write Better'], dtype=object)

In [14]:
len(products)

5

In [15]:
for i in products:
    train_1 = train[train['product']==i]
    #plt.subplot(len(products),1,j)
    #plt.subplot(1,2,len(i))
    plt.plot(train_1['date'],train_1['num_sold'])
    # Giving title to the chart using plt.title
    plt.title(f"Product '{i}' sold by Date")
    # rotating the x-axis tick labels at 30degree
    # towards right
    plt.xticks(rotation=30, ha='right')
 
 # Providing x and y label to the chart
    plt.xlabel('Date')
    plt.ylabel('Products sold')
    
    

In [16]:
train['date'].max()

Timestamp('2021-12-31 00:00:00')

In [17]:
train['date'].min()

Timestamp('2017-01-01 00:00:00')

In [18]:
import holidays

# Creating holiday calendars for each country
argentina_holidays = holidays.Argentina()
canada_holidays = holidays.Canada()
estonia_holidays = holidays.Estonia()
japan_holidays = holidays.Japan()
spain_holidays = holidays.Spain()

In [19]:
def new_column(df):
   df['day']=df['date'].dt.day
   df['month']=df['date'].dt.month
   df['year']=df['date'].dt.year
   df['is_argentina_holiday'] = df['date'].apply(lambda x: int(x in argentina_holidays))
   df['is_canada_holiday'] = df['date'].apply(lambda x: int(x in canada_holidays))
   df['is_estonia_holiday'] = df['date'].apply(lambda x: int(x in estonia_holidays))
   df['is_japan_holiday'] = df['date'].apply(lambda x: int(x in japan_holidays))
   df['is_spain_holiday'] = df['date'].apply(lambda x: int(x in spain_holidays))
   df['day_of_month'] = df.date.dt.day.astype("int8")
   df["quarter"] = df.date.dt.quarter.astype("int8")
   df['month_sin'] = np.sin(2*np.pi*df.month/12)
   df['month_cos'] = np.cos(2*np.pi*df.month/12)
   df['day_sin'] = np.sin(2*np.pi*df.day_of_month/31)
   df['day_cos'] = np.cos(2*np.pi*df.day_of_month/31)
   df=df.drop(['date'],axis=1)
   return df

In [20]:
train= new_column(train)

In [21]:
train.head()

,id,country,store,product,num_sold,day,month,year,is_argentina_holiday,is_canada_holiday,is_estonia_holiday,is_japan_holiday,is_spain_holiday,day_of_month,quarter,month_sin,month_cos,day_sin,day_cos
0,0,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63,1,1,2017,1,1,1,1,0,1,1,0.5,0.866025,0.201299,0.97953
1,1,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66,1,1,2017,1,1,1,1,0,1,1,0.5,0.866025,0.201299,0.97953
2,2,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9,1,1,2017,1,1,1,1,0,1,1,0.5,0.866025,0.201299,0.97953
3,3,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59,1,1,2017,1,1,1,1,0,1,1,0.5,0.866025,0.201299,0.97953
4,4,Argentina,Kaggle Learn,Using LLMs to Write Better,49,1,1,2017,1,1,1,1,0,1,1,0.5,0.866025,0.201299,0.97953


Exclude 2020 as it is outlier because of unpredictable covid 19 phase

In [22]:
train = train[train['year'] != 2020]

In [23]:
#stores=train['store'].unique()

In [24]:
X= train.drop(['num_sold'],axis=1)
y= train[['num_sold']]

In [25]:
cbe= CatBoostEncoder(cols = ['country', 'store', 'product']).fit(X,y)
X= cbe.transform(X)


In [26]:
y=np.log1p(y)

In [27]:
test['date']=pd.to_datetime(test['date'])

In [28]:
test=new_column(test)

In [29]:
test=cbe.transform(test)

In [30]:
def hgbr_objective(trial):
    # Use the trial object to suggest a value for alpha
    # Optuna will come up with a suggested value based on the scores of previous trials
    _alpha = trial.suggest_float("quantile", 0.1, 1)
    _beta = trial.suggest_float("learning_rate", 0.1, 20)
    _gamma = trial.suggest_int("max_iter", 1, 100)
    _delta= trial.suggest_int("max_leaf_nodes", 2, 100)
    _theta= trial.suggest_int("max_depth", 1, 12)
    # Define the model with the suggested alpha
    hgbr = HistGradientBoostingRegressor(quantile=_alpha,learning_rate=_beta,max_iter=_gamma,
                                         interaction_cst="pairwise",warm_start=1,l2_regularization=1,
                                         max_leaf_nodes= _delta,max_depth=_theta,random_state=42)
    # Calculate the score with 10-folds cross validation, which returns a list of scores
    # scoring is defined as negative RMSE as it is what this Kaggle competition uses to evaluate the result
    scores = cross_val_score(hgbr, X, y, 
                             cv=KFold(n_splits=10,
                                      shuffle=True,
                                      random_state=42),
                             scoring="neg_root_mean_squared_error"
                             #,error_score='raise'
                            )
    # Return the mean of 10 scores
    return scores.mean()

In [31]:
# Create Study object
study = optuna.create_study(direction="maximize")

[I 2023-07-25 07:42:35,633] A new study created in memory with name: no-name-dbb925d3-186c-4d2e-9593-67f0a593b1b5


In [32]:
# Optimize the study, use more trials to obtain better result, use less trials to be more cost-efficient
study.optimize(hgbr_objective, n_trials=200) # Use more

[I 2023-07-25 07:42:37,618] Trial 0 finished with value: -54129168021.276474 and parameters: {'quantile': 0.7963868800436467, 'learning_rate': 12.83173316281903, 'max_iter': 25, 'max_leaf_nodes': 45, 'max_depth': 7}. Best is trial 0 with value: -54129168021.276474.
[I 2023-07-25 07:42:44,969] Trial 1 finished with value: -0.07154362317160877 and parameters: {'quantile': 0.6394168685348411, 'learning_rate': 1.1809881177102282, 'max_iter': 90, 'max_leaf_nodes': 83, 'max_depth': 5}. Best is trial 1 with value: -0.07154362317160877.
[I 2023-07-25 07:42:46,938] Trial 2 finished with value: -1945698.635183656 and parameters: {'quantile': 0.43903290297273234, 'learning_rate': 5.252284392043122, 'max_iter': 75, 'max_leaf_nodes': 38, 'max_depth': 5}. Best is trial 1 with value: -0.07154362317160877.
[I 2023-07-25 07:42:48,471] Trial 3 finished with value: -3539233104738.036 and parameters: {'quantile': 0.3661914559100743, 'learning_rate': 19.61352974411506, 'max_iter': 32, 'max_leaf_nodes': 32,

In [33]:
# Print the result
best_params = study.best_params
best_score = study.best_value

In [34]:
best_score

-0.05653294310976818

In [35]:
hgbr= HistGradientBoostingRegressor(**best_params,random_state=42)

In [36]:
hgbr.fit(X,y)

HistGradientBoostingRegressor(learning_rate=1.1260929826290242, max_depth=12,
                              max_iter=92, max_leaf_nodes=99,
                              quantile=0.4008484970520073, random_state=42)

In [37]:
test.head()

,id,country,store,product,day,month,year,is_argentina_holiday,is_canada_holiday,is_estonia_holiday,is_japan_holiday,is_spain_holiday,day_of_month,quarter,month_sin,month_cos,day_sin,day_cos
0,136950,65.712922,61.909528,227.768216,1,1,2022,1,1,1,1,1,1,1,0.5,0.866025,0.201299,0.97953
1,136951,65.712922,61.909528,222.596169,1,1,2022,1,1,1,1,1,1,1,0.5,0.866025,0.201299,0.97953
2,136952,65.712922,61.909528,34.762737,1,1,2022,1,1,1,1,1,1,1,0.5,0.866025,0.201299,0.97953
3,136953,65.712922,61.909528,190.187786,1,1,2022,1,1,1,1,1,1,1,0.5,0.866025,0.201299,0.97953
4,136954,65.712922,61.909528,173.170344,1,1,2022,1,1,1,1,1,1,1,0.5,0.866025,0.201299,0.97953


In [38]:
prediction = hgbr.predict(test)

In [39]:
prediction=np.expm1(prediction)

In [40]:
output=pd.DataFrame({'id':test.id,'country':test.country,'num_sold':prediction})

In [41]:
output.head()

,id,country,num_sold
0,136950,65.712922,34.738218
1,136951,65.712922,30.332749
2,136952,65.712922,5.311239
3,136953,65.712922,24.140429
4,136954,65.712922,22.548276


In [42]:
temp_d= output.copy()

In [43]:
coef_df = 1/(temp_d.groupby('country')[['num_sold']].mean()/temp_d.groupby('country')[['num_sold']].mean().sum()/0.28)
coef_df

,num_sold
country,
65.712922,5.371949
132.383302,1.460675
164.753696,1.371148
221.840130,1.498781
263.795201,0.766791


In [44]:
temp_d['num_sold'] = temp_d.apply(lambda x: x['num_sold']*coef_df.iloc[0] if x['country']=='Argentina' else (
x['num_sold']*coef_df.iloc[4] if x['country']=='Spain' else (x['num_sold']*coef_df.iloc[3] if x['country']=='Japan' else (
x['num_sold']*coef_df.iloc[2] if x['country']=='Estonia' else x['num_sold']))), axis=1)

In [45]:
temp_d['num_sold'].dtype

dtype('float64')

In [46]:
output['num_sold']=temp_d['num_sold']

In [47]:
output.drop('country',axis=1)

,id,num_sold
0,136950,34.738218
1,136951,30.332749
2,136952,5.311239
3,136953,24.140429
4,136954,22.548276
...,...,...
27370,164320,729.256387
27371,164321,693.897111
27372,164322,82.316758
27373,164323,590.380272


In [48]:
output.head()

,id,country,num_sold
0,136950,65.712922,34.738218
1,136951,65.712922,30.332749
2,136952,65.712922,5.311239
3,136953,65.712922,24.140429
4,136954,65.712922,22.548276


In [49]:
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
